In [ ]:
from __future__ import absolute_import, division, print_function
import matplotlib.pyplot as plt
from chempy.chemistry import Reaction, Substance, ReactionSystem
from chempy.kinetics.ode import get_odesys
from chempy.util.graph import rsys2graph
import sympy
sympy.init_printing()
%matplotlib inline

In [ ]:
substances = A, B, C = map(Substance, 'ABC')
reactions = r0, r1, r2 = [
    Reaction({'A': 1}, {'B': 1}, .04),
    Reaction({'B': 1, 'C':1}, {'A': 1, 'C': 1}, 1e4),
    Reaction({'B': 2}, {'B': 1, 'C': 1}, 3e7)
]
rsys = ReactionSystem(reactions, substances)

In [ ]:
rsys2graph(rsys, 'robertson.png', save='.')
from IPython.display import Image
Image('robertson.png')

In [ ]:
odesys = get_odesys(rsys, include_params=True)
odesys.exprs

In [ ]:
odesys.get_jac()

In [ ]:
xout, yout, info = odesys.integrate(1e4, [1, 0, 0], integrator='cvode', first_step=1e-14)  # odeint may be better
for idx, lbl in enumerate('ABC'):
    plt.loglog(xout, yout[:, idx], label=lbl)
lgnd = plt.legend(loc='best')